In [1]:
import numpy as np
import plotly.graph_objs as go
import plotly.io as pio
import plotly.express as px

In [2]:
pio.templates.default = "plotly_white"

In [3]:
def build_line(pi,pf,stepSize):
    #Maximum travel ranges of the piezo stage
    #x,y = 100 um
    #z = 20 um
    #Defining the parametric equation of a line in R^3
    point_0 = pi #np.array([1,1,5])#Initial point
    point_f = pf # np.array([2,2,10])#Final point
    #line length
    line_len = np.linalg.norm(point_f - point_0) 
    print("Magnitud = ", line_len)
    step_size = stepSize # # Step size for line resolution : Also, resolution depends on laser pulse repetition and focus speed
    line_frac = step_size/line_len #Line fraction represented by the desiered stepSize
    t = np.linspace(0,1, int(np.ceil(1/line_frac)) + 1) # Defining all the desired fractions until complete the line 
    #Parametric line equations 
    x_lin = t * (point_f[0] - point_0[0]) + point_0[0]
    y_lin = t * (point_f[1] - point_0[1]) + point_0[1]
    z_lin = t * (point_f[2] - point_0[2]) + point_0[2]
    #Return x,y,z coordinates
    return x_lin, y_lin,z_lin



In [4]:
#Build a plane defining hatching distance ax + by + cz = D
#Plane with normal = <0,0,1>
def build_planes(x_0,x_f,y_0,y_f,z_0,z_f,hatching_dist,slicing_dist):
    #Define x y vectors
    #Define hatchin distance
    lenX = np.abs(x_f - x_0)
    lenY = np.abs(y_f - y_0)
    lenZ = np.abs(z_f - z_0)
    pointPerLine = int(np.ceil(lenX/hatching_dist)) + 1
    x_v = np.linspace(x_0, x_f, pointPerLine)
    y_v = np.linspace(y_0, y_f, int(np.ceil(lenY/hatching_dist)) + 1)
    #Define mesh grid 
    x_mesh,y_mesh = np.meshgrid(x_v,y_v)
    l, w = x_mesh.shape
    #constant matrix z 
    #z = z_0 * np.ones((l,w))
    #Define slicing distance and number of parallel planes 
    #Save the multiple planes in just 3 mattrices
    numberOfPlanes = int(np.ceil(lenZ/slicing_dist)) + 1
    multipleZVector = np.linspace(z_0, z_f, numberOfPlanes)
    #z matrix for the N planes 
    lenMatrix = l * w #lenMatrix : Defined by the dimensiones of a one plane matrix
    z_const_rows = np.tile(multipleZVector, (lenMatrix,1)) #Matrix with  z constant values per row
    x_flat = x_mesh.flatten().reshape(-1,1)
    y_flat = y_mesh.flatten().reshape(-1,1)
    x_data = np.tile(x_flat, (1, numberOfPlanes))
    y_data = np.tile(y_flat, (1, numberOfPlanes))
   # z = z_const_rows * np.ones((lenMatrix, numberOfPlanes))

    return x_data, y_data, z_const_rows, pointPerLine


In [5]:
def y_path_optimizer(y_matrix):
    y_matrix = y_matrix.copy()
    rows, cols = y_matrix.shape
    count = 1
    while count + 1 <= rows:
        y_matrix[count, :] = y_matrix[count, :][::-1]
        count += 2
    return y_matrix


In [6]:
#Optimzies the writing path for the x points 
#Function parameters: 
#pointNum: Number of points per line
#vector_opt: Vector containing the x data points
def x_path_optimizer(pointNum, vector_):
    vector_opt = vector_.copy()
    count = pointNum
    while count <= len(vector_opt) - pointNum:
        #Reverse the vector 
        vector_opt[count:count + pointNum] = vector_opt[count:count + pointNum][::-1]
        #New value for counter
        count += 2*pointNum
    #Return new vector 
    return vector_opt


In [7]:
#Function returns information of a plane per colum
#T transposes the data  (information of a plane per row)
x_data, y_data, z_data, points_per_line = build_planes(0, 12, 0, 12, 0, 8, hatching_dist = .5, slicing_dist = 1)
x_data = x_data.T
y_data = y_data.T
z_data = z_data.T


In [8]:
#Writing optimizers 
xOpt_data = x_path_optimizer(points_per_line,x_data.flatten())
yOpt_data = y_path_optimizer(y_data).flatten()
zOpt_data = z_data.flatten()
print(xOpt_data, "\n \n")
print(yOpt_data, "\n \n")
print(zOpt_data, "\n \n")
print(xOpt_data.size)


[0.  0.5 1.  ... 1.  0.5 0. ] 
 

[0. 0. 0. ... 0. 0. 0.] 
 

[ 0.  0.  0. ... 15. 15. 15.] 
 

10000


In [9]:
def flatten_to_matrix(x,y,z):
    #Concatenate the x,y,z data in order to visualize it with
    new_x = x.reshape(-1, 1)
    new_z = z.reshape(-1, 1)
    new_y = y.reshape(-1, 1)
    # Matrix representing the points to render (points per row)
    return np.concatenate((new_x, new_y, new_z), axis=1)


In [31]:
#Function that translate a matrix in the x axis
def x_translation(matrix, t):
    cMatrix = matrix.copy()
    x_t = cMatrix[:, 0] + t
    cMatrix[:, 0] = x_t
    return cMatrix


In [32]:
#Function that translate a matrix in the y axis
def y_translation(matrix, t):
    cMatrixY = matrix.copy()
    y_t = cMatrixY[:, 1] + t
    cMatrixY[:, 1] = y_t
    return cMatrixY


In [33]:
#Function that translate a matrix in the y axis
def z_translation(matrix, t):
    cMatrixZ = matrix.copy()
    z_t = cMatrixZ[:, 2] + t
    cMatrixZ[:, 2] = z_t
    return cMatrixZ


In [47]:
#translate the origing of the structure ... 
def origin_translation(matrix, x, y, z):
    new_matrix = matrix.copy()
    t1 = x_translation(new_matrix, x)
    t2 = y_translation(t1, y)
    return z_translation(t2, z)   


In [34]:
#Concatenate the x,y,z data in order to visualize it with 
points_matrix = flatten_to_matrix(xOpt_data,yOpt_data, zOpt_data) #Matrix representing the points to render (points per row)
points_matrix

array([[ 0. ,  0. ,  0. ],
       [ 0.5,  0. ,  0. ],
       [ 1. ,  0. ,  0. ],
       ...,
       [ 1. ,  0. , 15. ],
       [ 0.5,  0. , 15. ],
       [ 0. ,  0. , 15. ]])

In [87]:
import pandas as pd
df = pd.DataFrame(
    {"a": [1, 2, 3, 4], "b": [2, 1, 5, 6], "c": ["x", "x", "y", "y"]})

file = df.to_numpy()
np.savetxt("test_file.txt", points_matrix, delimiter="  ")


In [79]:
with open('test_file.txt') as f:
    lines = f.readlines()


In [82]:
lines

['0.000000000000000000e+00 0.000000000000000000e+00 0.000000000000000000e+00\n',
 '5.000000000000000000e-01 0.000000000000000000e+00 0.000000000000000000e+00\n',
 '1.000000000000000000e+00 0.000000000000000000e+00 0.000000000000000000e+00\n',
 '1.500000000000000000e+00 0.000000000000000000e+00 0.000000000000000000e+00\n',
 '2.000000000000000000e+00 0.000000000000000000e+00 0.000000000000000000e+00\n',
 '2.500000000000000000e+00 0.000000000000000000e+00 0.000000000000000000e+00\n',
 '3.000000000000000000e+00 0.000000000000000000e+00 0.000000000000000000e+00\n',
 '3.500000000000000000e+00 0.000000000000000000e+00 0.000000000000000000e+00\n',
 '4.000000000000000000e+00 0.000000000000000000e+00 0.000000000000000000e+00\n',
 '4.500000000000000000e+00 0.000000000000000000e+00 0.000000000000000000e+00\n',
 '5.000000000000000000e+00 0.000000000000000000e+00 0.000000000000000000e+00\n',
 '5.500000000000000000e+00 0.000000000000000000e+00 0.000000000000000000e+00\n',
 '6.000000000000000000e+00 0

In [89]:
with open('test_file.txt') as f:
    contents = f.read()
    print(type(contents))


<class 'str'>


In [48]:
origin_translation(points_matrix,1,2,3)

array([[ 1. ,  2. ,  3. ],
       [ 1.5,  2. ,  3. ],
       [ 2. ,  2. ,  3. ],
       ...,
       [ 2. ,  2. , 18. ],
       [ 1.5,  2. , 18. ],
       [ 1. ,  2. , 18. ]])

In [11]:

#print(xTest, "\n", yTest,z)
#Visualize the line
#Visualize the line
fig2 = go.Figure(go.Scatter3d(
    x=xOpt_data,
    y=yOpt_data,
    z=z_data.flatten(),
    mode='markers',
    marker=dict(
        size=1
    )

))

#fig1.update_layout(width = 600 , height= 600)
fig2.show()


In [128]:
def build_helix(heigth, r, numberofPoints, sliceDist):
    
    z_h = np.linspace(0, heigth, numberofPoints)
    t = ((2 * np.pi)/(sliceDist)) * z_h
    x_h = r * np.cos(t)
    y_h = r * np.sin(t)
    #z = t / ((2 * np.pi)/(sliceDist))
    return x_h, y_h, t


In [129]:
xL, yL, zL = build_line(np.array([1, 1, 5]), np.array([2, 2, 20]),1)
xL2, yL2, zL2 = build_line(np.array([2, 2, 5]), np.array([3, 3, 10]), .1)
xL2, yL2, zL2 = build_line(np.array([1.886, 7.774, 57.814]), np.array([3.323, 7.277, 63.974]), .1)
#Build helix
x_he, y_he, z_he = build_helix(16, 8, 10000, 1)


Magnitud =  15.066519173319364
Magnitud =  5.196152422706632
Magnitud =  6.344885972182633


In [91]:
line_matrix = flatten_to_matrix(xL,yL,zL)
#Concatenate different structures 
np.concatenate((points_matrix,line_matrix))

array([[ 0.    ,  0.    ,  0.    ],
       [ 0.5   ,  0.    ,  0.    ],
       [ 1.    ,  0.    ,  0.    ],
       ...,
       [ 1.875 ,  1.875 , 18.125 ],
       [ 1.9375,  1.9375, 19.0625],
       [ 2.    ,  2.    , 20.    ]])

In [130]:
#Visualize the line 
fig1 = go.Figure(go.Scatter3d(
    x=xL,
    y=yL,
    z=zL,
    mode = 'markers',
    marker = dict(
        size = 5
    )
))
fig1.add_traces(go.Scatter3d(
    x=xL2,
    y=yL2,
    z=zL2,
    mode='markers',
    marker=dict(
        size=5
    )
))
fig1.add_traces(go.Scatter3d(
    x=x_he,
    y=y_he,
    z=z_he,
    mode='markers',
    marker=dict(
        size=2
    )
))
#fig1.update_layout(width = 600 , height= 600)
fig1.show()


In [ ]:
import numpy as np
import pyvista as pv
from pyvista import examples


In [ ]:
dataset = examples.download_lidar()


In [ ]:
# Create random XYZ points
points = np.random.rand(100, 3)
# Make PolyData
point_cloud = pv.PolyData(points)
